In [ ]:
!pip install transformers
!pip install sentence_transformers
!pip install wtpsplit
!pip install accelerate
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Imports

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from transformers import AutoTokenizer, LongformerTokenizer, LongformerModel, LongformerForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
from wtpsplit import SaT
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from datasets import load_dataset, load_from_disk
import accelerate

# Setup
Initializes variables and loads data as dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")
dataset_folder = '/content/drive/MyDrive/paper-moderation-SP25'
output_folder = "/content/drive/MyDrive/paper-moderation-SP25/longformer"
os.makedirs(output_folder, exist_ok=True)

train_dataset = load_from_disk(dataset_folder + "/new_train.arrow")
val_dataset = load_from_disk(dataset_folder + "/new_val.arrow")
test_dataset = load_from_disk(dataset_folder + "/new_test.arrow")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model Training

In [ ]:
def generate_tokens(data):
    """
    Train a Longformer classifier on the prepared datasets.

    Args:
        data: Input data
    Returns:
        Tokens generated from the input data
    """
    return tokenizer(
    data["text"],
    padding="max_length",
    truncation=True,
    max_length=4096,
    )

def train_classifier(train_dataset, val_dataset, test_dataset):
    """
    Train a Longformer classifier on the prepared datasets.

    Args:
        train_dataset: Training dataset
        test_dataset: Testing dataset
        val_dataset: Validation dataset
        num_classes: Number of classes for classification

    Returns:
        Trained classifier model
    """
    print("Training classifier model...")

    # Load the classification model
    classifier = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096").to(device)

    training_args = TrainingArguments(
    output_dir=f"{output_folder}/base_model",
    save_strategy="epoch",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    fp16=True,
    report_to="none",
    )

    # Define metrics for evaluation
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return {
            # other metrics: mainly precision
            "accuracy": accuracy_score(labels, predictions),
            "precision": precision_score(labels, predictions),
            "recall": recall_score(labels, predictions)
        }

    # Create trainer
    trainer = Trainer(
        model=classifier,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    # Train the model
    print("Starting training...")
    trainer.train()

    # Evaluate the model
    print("Evaluating model...")
    eval_results = trainer.evaluate(test_dataset)
    print(f"Evaluation results: {eval_results}")

    return classifier

# Main
Run this to execute the entire pipeline

In [ ]:
def main(max_papers_per_source=None):
    """
    Main function to orchestrate the entire pipeline.
    """
    print("Starting paper processing pipeline...")

    #Apply tokenization to all datasets
    print("Starting data tokenization...")
    tokenized_train = train_dataset.map(generate_tokens, batched=True)
    tokenized_val = val_dataset.map(generate_tokens, batched=True)
    tokenized_test = test_dataset.map(generate_tokens, batched=True)

    tokenized_train.save_to_disk(f"{output_folder}/base_datasets/train")
    tokenized_val.save_to_disk(f"{output_folder}/base_datasets/val")
    tokenized_test.save_to_disk(f"{output_folder}/base_datasets/test")
    print("Datasets saved to Google Drive.")

    print("Tokenization of train dataset complete.")

    # Format datasets to return PyTorch tensors
    tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

    classifier = train_classifier(tokenized_train, tokenized_val, tokenized_test)

    print("Pipeline complete!")

In [ ]:
main()

Starting paper processing pipeline...
Starting data tokenization...


Map:   0%|          | 0/27499 [00:00<?, ? examples/s]

Map:   0%|          | 0/9166 [00:00<?, ? examples/s]

Map:   0%|          | 0/9166 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/27499 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/597M [00:00<?, ?B/s]

Step,Training Loss
500,0.305200
1000,0.199200
1500,0.131500
2000,0.100800
2500,0.073000


Step,Training Loss
500,0.305200
1000,0.199200
1500,0.131500
2000,0.100800
2500,0.073000
3000,0.064800


Evaluating model...


Evaluation results: {'eval_loss': 0.06556926667690277, 'eval_accuracy': 0.9853807549639974, 'eval_precision': 0.9770196635868278, 'eval_recall': 0.9911079067531843, 'eval_runtime': 1738.7991, 'eval_samples_per_second': 5.271, 'eval_steps_per_second': 0.659, 'epoch': 0.9998545454545454}
Pipeline complete!


# Tokenized Data
Run this once data has been tokenized and saved to output folder.

In [ ]:
"""
This loads everything that has been tokenized to retrieve the accuracy of the model
"""
def run_pipeline():
    tokenized_train = load_from_disk(f"{output_folder}/base_datasets/train")
    tokenized_val = load_from_disk(f"{output_folder}/base_datasets/val")
    tokenized_test = load_from_disk(f"{output_folder}/base_datasets/test")

    # Format datasets to return PyTorch tensors
    tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

    # Train
    model = train_classifier(tokenized_train, tokenized_val, tokenized_test)

In [ ]:
run_pipeline()

Training classifier model...


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting training...


Step,Training Loss
500,0.108900
1000,0.066300
1500,0.030400
2000,0.022200
2500,0.018400
